In [622]:
import random
import jieba.analyse
import pandas as pd

## 1. 加载停用词

In [623]:
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\百度停用词表.txt"
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\哈工大停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/哈工大停用词表.txt"
stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/中文停用词表.txt"
stop_words = pd.read_csv(stop_words_path,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stop_words = stop_words['stopword'].values

## 2. 加载语料

In [624]:
# data_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\面试-训练测试集.xlsx"
data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-训练测试集.xlsx"
data = pd.read_excel(data_path)

# title_data = data['title']
# content_data = data['content']

## 3.定义文本预处理函数

* 去停用词，分词
* 参数 titles, contents, sentences, labels

In [625]:
def is_float(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
    return False

def is_percents(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        right = right.split('%')[0]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
            
    elif s.count(".") != 1:
        left = s.split('%')[0]
        if left.isdigit():
            return True
               
    return False


def preprocess_text(title, content, sentences, label):
    
#     处理标题
    title_info = jieba.lcut(title)

    title_info = [v for v in title_info if not str(v).isdigit()]#去数字
    title_info = list(filter(lambda x:x.strip(), title_info))   #去左右空格
    

    title_info = list(filter(lambda x:len(x)>1, title_info)) #长度为1的字符
    title_info = list(filter(lambda x:x not in stop_words, title_info)) #去掉停用词
    
#     print("标题信息是：" ,title_info)
#     sentences.append(segs)
#     sentences.append((" ".join(segs), label))# 打标签
            
#     处理文章内容        
    segs=jieba.lcut(content)
    segs = [v for v in segs if not str(v).isdigit()]#去数字
    segs = [v for v in segs if not is_float(v)] #去小数
    segs = [v for v in segs if not is_percents(v)] #去百分数

    segs = list(filter(lambda x:x.strip(), segs))   #去左右空格
    segs = list(filter(lambda x:len(x)>1, segs)) #长度为1的字符
    segs = list(filter(lambda x:x not in stop_words, segs)) #去掉停用词
    
#     print("内容信息是:", segs)
    segs = title_info + segs
    sentences.append((" ".join(segs), label))# 打标签

    


## 4. 文本预处理

In [626]:
sentences = []

for index, row in data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    label = row['label']
    preprocess_text(title, content, sentences, label)
    

In [627]:
# random.shuffle(sentences)
for sentence in sentences[:10]:
    print("-------")
    print("文章分词:", sentence[0])
    print("文章标签：", sentence[1])

-------
文章分词: 赵东 交易所 BTC 周线 放量 现在 反弹 牛市 DGroup 创始人 赵东 发微 博称 激动 交易所 BTC 周线 放量 现在 反弹 牛市 牛会来 现在 耐心
文章标签： 正面
-------
文章分词: 世界 行情 BTC 小幅 回调 美元 附近 宽幅 震荡 BTC 昨日 高涨 美元 目前 小幅 回调 主流 多数 小幅 回调 BTC 在币安现 美元 24h 跌幅 全球 数字 货币 市场 总价值 亿美元 24h 成交量 亿美元 BTC 全球 均价 美元 三大 主流 交易所 火币 现报 美元 OKEx 现报 美元 币安现 美元 主流 数字 货币 表现 ETH 暂报 美元 XRP 暂报 美元 BCH 暂报 美元 LTC 暂报 美元 ETC 暂报 美元 EOS 暂报 美元 小时 行情 市值 排名 前百 币种 涨幅 前三为 NULS NANO XTZ 跌幅 前三为 FT WAX PAI 世界 行情 概念 板块 涨幅 前三为 DAG 技术 合约 工具 跨链 跌幅 前三为 人工智能 身份验证 联网 涨跌幅 24h 计算
文章标签： 正面
-------
文章分词: BTC 短时 跌破 美元 世界 行情 BTC 刚刚 快速 下跌 短时 跌破 美元 最低 跌至 美元 略有 回升 BTC 火币 现报 美元 今日 跌幅 实时 行情 异动 提醒 点击 查看 原文 开启 智能 盯盘
文章标签： 负面
-------
文章分词: 过去 小时 推特 讨论 BTC 排名 第一 ETH XRP 排名 三位 CoinTrendz com 数据 显示 过去 小时 推特 讨论 排行 BTC 讨论 排名 第一 ETH 排名 第二位 XRP 排名 第三位 讨论 排名 四至 十位 LTC BAT ADA ATT TRX XVG BNB
文章标签： 负面
-------
文章分词: Coindesk 分析 BTC 美元 均线 支撑 有效 Coindesk 分析 BTC 未来 几天 可能 回落 美元 第一季度 引领 市场 走高 LTC 显示 疲软 迹象 中旬 月底 LTC BTC 几乎 走势 相同 BTC 多头 空头 比率 首次 跌至 以下 表明 看跌 情绪 日益加剧 价格 回调 看起来 可能 BTC 价格 借助 上升 移动 平均线 目前 美元 反弹 下跌 美元 情况 将会 减弱


## 5. 抽取词向量特征

In [628]:
# 1. 抽取特征， 定义词袋模型

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vec = CountVectorizer(
#     analyzer='word', # tokenise by character ngrams
#     ngram_range=(1,4),  # use ngrams of size 1 and 2
#     max_features=1000,  # keep the most common 1000 ngrams
# )


vec = TfidfVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,2),  # use ngrams of size 1 and 2
    max_features=500,  # keep the most common 1000 ngrams
)

In [629]:
# 2. 语料切分

from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.35, random_state=1256)
# print(y_train)

In [630]:
# 3. 把训练数据转为词袋模型
vec.fit(x_train)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=500,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [631]:
# 4.算法建模与模型训练 （贝叶斯）
# from sklearn.naive_bayes import MultinomialNB
# classifier = MultinomialNB()
# classifier.fit(vec.transform(x_train), y_train)


In [632]:
# 5. 评估结果 AUC值
# print(classifier.score(vec.transform(x_test), y_test))


In [633]:
# 4.算法建模与模型训练 （SVM）

from sklearn.svm import SVC
# svm = SVC(kernel='linear')
svm = SVC(C=1.0,kernel='linear',gamma='auto')
svm.fit(vec.transform(x_train), y_train)

print("SVM模型评价，训练集正确率",svm.score(vec.transform(x_train), y_train))
print("SVM模型评价，测试集正确率",svm.score(vec.transform(x_test), y_test))

SVM模型评价，训练集正确率 0.8796296296296297
SVM模型评价，测试集正确率 0.8685714285714285


## 目前准确率在测试集上为80%左右

In [634]:
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='relu',solver='lbfgs',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.001,max_iter=2000)
model.fit(vec.transform(x_train),y_train)

print("神经网络模型评价，训练集正确率", model.score(vec.transform(x_train), y_train))

acc=model.score(vec.transform(x_test),y_test) #根据给定数据与标签返回正确率的均值
print('神经网络模型评价, 测试集正确率:',acc)

神经网络模型评价，训练集正确率 0.9969135802469136
神经网络模型评价, 测试集正确率: 0.8171428571428572
